# Obtenemos los datos

Aquí descargamos los datos desde kaggle. Desde kaggle descargamos un token con los permisos de la api de kaggle. Al momento de iniciar solamente subimos el token a nuestros archivos en colab, asignamos el directorio y usamos unas funciones de kaggle para descargar el dataset de la competición, posteriormente solo le hacemos un unzip a las carpetas con las imagenes para tenerlas descomprimidas y poder trabajar con ellas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/"
!pwd

Mounted at /content/drive
/content
/content


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download hotel-id-to-combat-human-trafficking-2022-fgvc9

100% 14.0G/14.0G [02:12<00:00, 131MB/s]
100% 14.0G/14.0G [02:13<00:00, 113MB/s]


In [ ]:
! unzip -q hotel-id-to-combat-human-trafficking-2022-fgvc9 

Importamos las librerías que son necesarias para el preprocesamiento de los datos que se desean realizar.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time

import shutil
from shutil import move
from random import sample
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from random import sample

Seleccionamos los directorios y podemos ver que se tienen 3116 carpetas, es decir, de nuestro set de datos hay 3116 clases.

In [ ]:
base_dir = '/content/'
train_dir = os.path.join(base_dir,'train_images')
chain_names = os.listdir(train_dir)

print(len(chain_names))


3116


Usamos una función que nos permite crear un folder.

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## Seleccionamos las carpetas que tengan más de 30 imagenes

In [ ]:
validation_data_dir = os.path.join(base_dir, 'test_images')
masks_dir = os.path.join(base_dir, 'train_masks')

A continuación filtramos todas las categorías a solo aquellas que contengan más de 30 imagenes. Todas aquellas con 30 o menos consideramos que no tienen información suficiente y, para obtener un mejor resultado de los modelos, decidimos no utilizarlas.

In [ ]:
from os import listdir
from os.path import isfile, join
from random import sample
min_len = 30
train_set = [i for i in listdir(train_dir) if len(listdir(os.path.join(train_dir, i))) > min_len]
train_len = [len(listdir(os.path.join(train_dir, i)))for i in listdir(train_dir) if len(listdir(os.path.join(train_dir, i))) > min_len]
print('Tenemos en total: ', len(train_set), ' conjuntos')
print('Con un total de ', sum(train_len), ' imagenes')
print('Siendo el {:.2f}% de las imagenes totales'.format(sum(train_len)/sum([len(listdir(os.path.join(train_dir, i))) for i in listdir(train_dir)])*100))


Tenemos en total:  174  conjuntos
Con un total de  13833  imagenes
Siendo el 30.94% de las imagenes totales


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
final_train = os.path.join(base_dir, 'final_train')

In [ ]:
final_train

'/content/final_train'

In [ ]:
createFolder('final_train')

Aquí usamos el nuevo folder 'final_train' en el cual hacemos unas copias de las carpetas que cumplen con el filtro anterior, además también tenemos la opción de hacer que si una carpeta tiene k+1 imagenes o más, se hace un submuestreo aleatorio de k imagenes de entre todas ellas. Al final decidimos no utilizar esto y trabajar con el desbalanceo de otra manera, pero la opción sigue ahí.

In [ ]:
for folder in train_set:
    path = os.path.join(train_dir, folder)
    new_path = os.path.join(final_train, folder)
    os.mkdir(new_path)
    images = [f for f in os.listdir(path)]
    # cancelamos lo siguiente para que no haga muestreo de las imagenes
    '''if len(images)>=90:
      images = sample(images, k=90)'''
    for image in images:
      shutil.copy(os.path.join(path, image), new_path)

In [ ]:
classes = len(train_set)

In [ ]:
#probar batch hasta el 30% de las clases
train_generator = train_datagen.flow_from_directory(
    final_train,
    target_size=(150, 150),
    batch_size=int(classes*0.35),
    class_mode='categorical',
    color_mode = 'rgb',
    shuffle = True
)

Found 13833 images belonging to 174 classes.


In [ ]:
final_train_len = [len(listdir(os.path.join(final_train, i)))for i in listdir(final_train)]

### Validation

En esta parte generamos el conjunto de validación. Para poder hacerlo simplemente hacemos un submuestreo del 15% de las imagenes de cada clase y las copiamos en la carpeta nueva 'validation', esta se utilizara como validación al entrenar la red.

In [ ]:
createFolder('validation')

In [ ]:
validation_dir = os.path.join('/content/validation')

In [ ]:
proportion = 0.15

for folder in os.listdir(final_train):
    path = os.path.join(final_train, folder)
    new_path = os.path.join(validation_dir, folder)
    os.mkdir(new_path)
    images = [f for f in os.listdir(path)]
    images = sample(images, k=int(proportion*len(images))) 
    for image in images:
        shutil.copy(os.path.join(path, image), new_path)

In [ ]:
#shutil.rmtree(validation_dir)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_generator = validation_datagen.flow_from_directory(
																										validation_dir,
                                                    target_size=(150, 150),
                                                    batch_size=int(classes*0.35),
                                                    class_mode='categorical',
                                                    color_mode = 'rgb',
                                                    shuffle = True)

Found 1990 images belonging to 174 classes.


Finalmente hacemos zip a las dos carpetas anteriores, lo cual nos permitira descargarlas y entrenar la red localmente. Esto debido a que nos dimos cuenta que de forma local, el sobremuestreo que haremos después se hace mucho más rápido.

In [ ]:
!zip -r -q /content/final_train.zip /content/final_train

In [ ]:
  !zip -r -q /content/validation.zip /content/validation

In [ ]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()

In [ ]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)

    created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created